In [1]:
import os
from os import sys
sys.path.append('./data')
import numpy as np
import matplotlib.pyplot as plt
from species import *

All variables set as e.g. krome_idx_H2
plot 'your_output' u 1:krome_idx_H2
 the offset is nkrome= 0


In [2]:
os.system('clear')
os.system('make gfortran')

512

In [3]:
def Av(NH):
    return ('{:.2f}d0'.format(NH * 5.3e-22))


def write_test_Xray(NH, t=0):
    with open('./test.f90', 'r') as f:
        lines = f.readlines()
        lines[20] = '  call krome_set_user_av({}) !opacity Av (#)\n'.format(
            Av(NH))
        if t > 0:
            lines[
                45] = '  call krome(x(:),Tgas,{:.1e}*spy) !call KROME before the AGN event\n'.format(
                    t)
        else:
            lines[45] = '  \n'
        lines[49] = '  call krome_set_J21xray(1d0)\n'
        lines[50] = '  open(unit=77, file="./data/dis")\n'
        lines[68] = '  call krome_set_J21xray(1d0)\n'
    with open('./test.f90', 'w') as f:
        for l in lines:
            f.write(l)


def write_test_stop(NH, t=0):
    with open('./test.f90', 'r') as f:
        lines = f.readlines()
        lines[20] = '  call krome_set_user_av({}) !opacity Av (#)\n'.format(
            Av(NH))
        if t > 0:
            lines[
                45] = '  call krome(x(:),Tgas,{:.1e}*spy) !call KROME before the AGN event\n'.format(
                    t)
        else:
            lines[45] = '  \n'
        lines[49] = '  call krome_set_J21xray(1d0)\n'
        lines[50] = '  open(unit=77, file="./data/dis")\n'
        lines[68] = '  call krome_set_J21xray(0d0)\n'
    with open('./test.f90', 'w') as f:
        for l in lines:
            f.write(l)


def write_test0(NH, t=0):
    with open('./test.f90', 'r') as f:
        lines = f.readlines()
        lines[20] = '  call krome_set_user_av({}) !opacity Av (#)\n'.format(
            Av(NH))
        if t > 0:
            lines[
                45] = '  call krome(x(:),Tgas,{:.1e}*spy) !call KROME before the AGN event\n'.format(
                    t)
        else:
            lines[45] = '  \n'
        lines[49] = '  call krome_set_J21xray(0d0)\n'
        lines[50] = '  open(unit=77, file="./data/dis_inf")\n'
        lines[68] = '  call krome_set_J21xray(0d0)\n'
    with open('./test.f90', 'w') as f:
        for l in lines:
            f.write(l)

In [4]:
def AGN(NHs=[10**22.5], Dis=[4], t_before=0):
    t_mark = ''
    if t_before > 0:
        t_mark = '{:.0e}'.format(t_before)
    for NH in NHs:
        with open('krome_getphys.f90', 'r') as f:
            lines = f.readlines()
            lines[3715] = 'num2col = max(ncalc,1d-40)*{}\n'.format(
                str(NH / 2e4))
        with open('krome_getphys.f90', 'w') as f:
            for l in lines:
                f.write(l)
        label = str(int((np.log10(NH) - 22) * 2 + 1))
        for dis in Dis:
            nH = './rate_dis/ratexH' + str(dis) + '.dat'
            nHe = './rate_dis/ratexHe' + str(dis) + '.dat'
            orderH = 'cat ' + nH + ' > ratexH.dat'
            os.system(orderH)
            orderHe = 'cat ' + nHe + ' > ratexHe.dat'
            os.system(orderHe)

            write_test_Xray(NH=NH, t=t_before)
            os.system('make gfortran')
            os.system('./test')
            command = 'cat ./data/dis > ./data/{}dis{}{}'.format(
                str(label), str(dis), t_mark)
            os.system(command)
            command = 'cat ./data/Trace > ./data/{}Trace{}{}'.format(
                str(label), str(dis), t_mark)
            os.system(command)
            command = 'rm ./data/dis & rm ./data/Trace'
            os.system(command)
            cowsay = 'cowsay -f www I am ionized at ' + str(dis) + ' kpc'
            os.system(cowsay)

            write_test_stop(NH=NH, t=t_before)
            os.system('make gfortran')
            os.system('./test')
            command = 'cat ./data/dis > ./data/{}dis0{}{}'.format(
                str(label), str(dis), t_mark)
            os.system(command)
            command = 'cat ./data/Trace > ./data/{}Trace0{}{}'.format(
                str(label), str(dis), t_mark)
            os.system(command)
            command = 'rm ./data/dis & rm ./data/Trace'
            os.system(command)
            cowsay = 'cowsay -f www I am ionized at ' + str(
                dis) + ' kpc, but have been saved at 1E6 YEARS'
            os.system(cowsay)

        command = 'rm ./data/dis ./data/Trace'.format(str(label))
        os.system(command)

        write_test0(NH=NH, t=t_before)
        os.system('make gfortran')
        os.system('./test')
        command = 'cat ./data/dis_inf > ./data/{}dis_inf{}'.format(
            str(label), t_mark)
        os.system(command)
        command = 'cat ./data/Trace > ./data/{}Trace_inf{}'.format(
            str(label), t_mark)
        os.system(command)
        command = 'rm ./data/dis_inf & rm ./data/Trace'
        os.system(command)
        os.system('cowsay -f www I feel ok')

In [5]:
AGN() # Default Model, inside both situation below

In [6]:
AGN(Dis=np.arange(8) + 1) # Galactic Distribution

In [7]:
AGN(NHs=[1e22, 1e23]) # Distribution inside

In [8]:
AGN(t_before=1e4)
AGN(t_before=1e7)